In [1]:
import sys

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core

In [2]:
from morpheus.project.types.discretization.spatial import Grid, LengthUnit, Rotation
from morpheus.project.types.geometry import Polygon

polygon = Polygon(
    type='Polygon',
    coordinates=[
        [
            (13.922514437551428, 50.964720483303836),
            (13.925250781947113, 50.965228748412386),
            (13.925036413951403, 50.96623732041704),
            (13.92222441026388, 50.96609040370362),
            (13.922514437551428, 50.964720483303836),
        ]
    ],
)

rotation = Rotation(20)
length_unit = LengthUnit.meters()
relative_col_coordinates = [0, 0.1, 0.21, 0.22, 0.23, 0.24, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
relative_row_coordinates = [0, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 1]

grid = Grid.from_polygon_with_relative_coordinates(
    polygon=polygon,
    relative_col_coordinates=relative_col_coordinates,
    relative_row_coordinates=relative_row_coordinates,
    rotation=rotation,
)
polygon

Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96609040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon')

In [3]:
grid.get_wgs_cell_center_coordinates()

([[13.92201083954258,
   13.92232594809525,
   13.922506010125344,
   13.922536020463697,
   13.922566030802045,
   13.922596041140395,
   13.922686072155441,
   13.922911149693062,
   13.923211253076557,
   13.923511356460049,
   13.923811459843543,
   13.924111563227036,
   13.92441166661053,
   13.924711769994024],
  [13.922094182248262,
   13.922409290800932,
   13.922589352831025,
   13.922619363169376,
   13.922649373507726,
   13.922679383846075,
   13.922769414861124,
   13.922994492398745,
   13.923294595782238,
   13.923594699165731,
   13.923894802549226,
   13.924194905932715,
   13.92449500931621,
   13.924795112699703],
  [13.922102119648805,
   13.922417228201471,
   13.922597290231568,
   13.92262730056992,
   13.922657310908269,
   13.922687321246618,
   13.922777352261665,
   13.923002429799286,
   13.923302533182781,
   13.923602636566272,
   13.923902739949765,
   13.924202843333259,
   13.924502946716752,
   13.924803050100246],
  [13.922110057049345,
   13.9224251

In [4]:
grid.get_wgs_bbox()

(13.921781413845427, 50.964615422168436, 13.92557618773444, 50.96667675655003)

In [5]:
import folium

m = folium.Map(crs='EPSG3857', zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [6]:
m.add_child(folium.GeoJson(grid.get_wgs_outline_geometry().as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [7]:
cell_geometries = grid.get_wgs_cell_geometries()

n_x = grid.n_cols()
n_y = grid.n_rows()

for col in range(n_x):
    for row in range(n_y):
        cell_geometry = cell_geometries[row][col]
        folium.GeoJson(cell_geometry.as_geojson()).add_to(m)

m

In [8]:
# Render GRid Rows and Columns
from morpheus.project.types.discretization.spatial import ActiveCells

cells = ActiveCells.from_polygon(polygon, grid)

m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson(), name='hello world').add_to(m)
folium.GeoJson(grid.to_geojson().as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [9]:
from morpheus.project.types.discretization.spatial import ActiveCells

cells_geometries = ActiveCells.from_polygon(polygon, grid).to_geojson(grid)

m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson(), name='hello world').add_to(m)
folium.GeoJson(grid.get_wgs_outline_geometry().as_geojson()).add_to(m)
folium.GeoJson(cells_geometries.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [10]:
cells_outline = cells.outline_to_geojson(grid)

m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson()).add_to(m)
folium.GeoJson(grid.get_wgs_outline_geometry().as_geojson()).add_to(m)
folium.GeoJson(cells_outline.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())

m